In [ ]:
import pandas as pd
import numpy as np

In [ ]:
class Normalization:
    def __init__(self, matrix, cb, weights):
        if matrix.shape[1] != len(weights) or matrix.shape[1] != len(cb) or len(cb) != len(weights):
            raise ValueError(f'Data shape, cost-benefit vector or weights vector does not match')
        self.matrix = matrix
        self.cb = cb
        self.weights = weights

    def normalization_zero_one(self):
        result = self.matrix.copy()
        pos = 0
        for col in self.matrix.columns:
            max_value = self.matrix[col].max()
            min_value = self.matrix[col].min()
            dif_max_min = self.matrix[col].max() - df[col].min()
            if self.cb[pos] == 'benefit':
                result[col] = (self.matrix[col] - min_value) / dif_max_min
            else:
                result[col] = (max_value - self.matrix[col]) / dif_max_min
            pos+=1
        return result.round(4)

In [ ]:
def promethee_ii(df, weights, cb):
    nrow = df.shape[0]
    ncol = df.shape[1]
    # Normalization
    ndm = Normalization(df, cb, weights).normalization_zero_one()
    # Pairwise comparisons
    pc_matrix = pd.DataFrame(np.zeros((nrow**2, ncol)))
    aux=0
    for r in range(ndm.shape[0]):
        for s in range(ndm.shape[0]):
            pc_matrix.iloc[aux][:] = ndm.iloc[r, :] - ndm.iloc[s, :]
            aux+=1

    # Calculate the preference function
    pref_func_matrix = pc_matrix.copy()
    pref_func_matrix[pref_func_matrix < 0] = 0

    # Calculate the aggregated preference function
    agg_pref_func_matrix = (pref_func_matrix * weights).apply(sum, axis=1)
    agg_pref_func_matrix = pd.DataFrame(agg_pref_func_matrix.values.reshape(nrow, nrow))

    # Determine the leaving and entering outranking flows
    outranking_flows = pd.DataFrame(np.zeros((nrow, 2)), columns=['Leaving','Entering'])
    outranking_flows['Leaving'] = agg_pref_func_matrix.apply(sum, axis=1)/(ncol-1)
    outranking_flows['Entering'] =  agg_pref_func_matrix.apply(sum, axis=0)/(ncol-1)
    outranking_flows['Leav_Enter'] = outranking_flows['Leaving'] - outranking_flows['Entering']

    # Determine the ranking
    outranking_flows['Rank'] = outranking_flows['Leav_Enter'].rank(ascending=False).astype(int)

    return pc_matrix, outranking_flows['Rank'].to_list()

In [ ]:
df = pd.DataFrame({'Price': [250, 200, 300, 275],
                   'Storage': [16, 16, 32, 32],
                   'Camera': [12, 8, 16, 8],
                   'Looks': [5, 3, 4, 2]})

In [ ]:
weights = [.35, .25, .25, .15]
cb = ['cost', 'benefit', 'benefit', 'benefit']

In [ ]:
pc_matrix, rank = promethee_ii(df, weights,cb)

In [ ]:
print(pc_matrix)

       0    1    2       3
0   0.00  0.0  0.0  0.0000
1  -0.50  0.0  0.5  0.6667
2   0.50 -1.0 -0.5  0.3333
3   0.25 -1.0  0.5  1.0000
4   0.50  0.0 -0.5 -0.6667
5   0.00  0.0  0.0  0.0000
6   1.00 -1.0 -1.0 -0.3334
7   0.75 -1.0  0.0  0.3333
8  -0.50  1.0  0.5 -0.3333
9  -1.00  1.0  1.0  0.3334
10  0.00  0.0  0.0  0.0000
11 -0.25  0.0  1.0  0.6667
12 -0.25  1.0 -0.5 -1.0000
13 -0.75  1.0  0.0 -0.3333
14  0.25  0.0 -1.0 -0.6667
15  0.00  0.0  0.0  0.0000


In [ ]:
print(rank)

[2, 3, 1, 4]
